$$\Huge \text{RAPORT}$$
$$\huge \text{LISTA 2}$$
$$\large \text{Autorzy: Katarzyna Rudzińska, Jagoda Płócienniczak}$$

## CZĘŚĆ I

### ZADANIE 1
    W ankiecie przedstawionej na poprzedniej liście pracownicy zostali poproszeni o wyrażenie opinii na temat skuteczności szkolenia "Efektywna komunikacja w zespole" zorganizowanego przez firmę. Wśród próbki 200 pracowników (losowanie proste ze zwracaniem) uzyskano wyniki:
    • 14 pracowników - bardzo niezadowolonych,
    • 17 pracowników - niezadowolonych,
    • 40 pracowników - nie ma zdania,
    • 100 pracowników - zadowolonych,
    • 29 pracowników - bardzo zadowolonych,
    Na podstawie danych wyznacz przedział ufności dla wektora prawodobieństw opisującego stopień zadowolenia ze szkolenia. Przyjmij poziom ufności 0,95.

### ZADANIE 2 
    Napisz funkcję, która wyznacza wartość poziomu krytycznego w następujących
    testach:
        • chi-kwadrat Pearsona
        • chi-kwadrat największej wiarogodności
    służących do weryfikacji hipotezy H0 : p = p0 przy hipotezie alternatywnej H1 : p = p0 na podstawie obserwacji x wektora losowego X z rozkładu wielomianowego z parametrami n i p.

### ZADANIE 3
    Na podstawie danych z ankiety z poprzedniej listy zweryfikuj hipotezę, że w grupie
    pracowników zatrudnionwych w Dziale Produktowym rozkład odpowiedzi na pytanie "Jak
    bardzo zgadzasz się ze stwierdzeniem, że firma zapewnia odpowiednie wsparcie i materiały
    umożliwiające skuteczne wykorzystanie w praktyce wiedzy zdobytej w trakcie szkoleń?" jest
    równomierny, tzn. jest jednakowe prawdopodobieństwo, że pracownik zatrudniony w Dziale Produkcjnym udzielił odpowiedzi "zdecydowanie się nie zgadzam", "nie zgadzam się", "nie
    mam zdania", "zgadzam się", "zdecydowanie się zgadzam" na pytanie PYT_1. Przyjmij poziom
    istotności 0,05. Skorzystaj z funkcji napisanej w zadaniu 2.

## CZĘŚĆ II

### ZADANIE 4
    Zapoznaj się z funkcjami służącymi do wykonania testu Fishera oraz testu Freemana-Haltona.

### ZADANIE 5
    Korzystajac z testu Fishera, na poziomie istotnosci 0.05, zweryfikuj hipotezę, że zmienna 'PŁEć' i zmienna 'CZY_KIER' są niezależne. Czy na poziomie istotności 0.05 możemy wnioskować, że prawdopodobieństwo tego, że na stanowisku kierowniczym pracuje kobieta jest równe prawdopodobieństwu tego, że na stanowisku kierowniczym pracuje mężczyzna? Uzasadnij odpowiedź.

### ZDANIE 6
    Korzystajac z testu Freemana-Haltona na poziomie istotności 0,05 zweryfikuj następujące hipotezy:
        a) zajmowanie stanowiska kierowniczego nie zależy od wieku (CZY_KIER oraz WIEK_KAT),
        b) zajmowanie stanowiska kierowniczego nie zależy od stażu pracy (CZY_KIER oraz STAŻ),
        c) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w pierwszym badanym okresie nie zależy od zajmowanego stanowiska (PYT_2 oraz CZY_KIER),
        d) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w pierwszym badanym okresie nie zależy od stażu (PYT_2 oraz STAŻ),
        e) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w pierwszym badanym okresie nie zależy od płci (PYT_2 oraz PŁEĆ),
        f) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w pierwszym badanym okresie nie zależy od wieku (PYT_2 oraz WIEK_KAT).
    Wykonaj podpunkty c), d), e) oraz d) zast˛ epuj˛ ac zmienn˛ a PYT_2 przez zmienną CZY_ZADOW zdefiniowaną w zadaniu 1. w części I z poprzedniej listy. Porównaj wyniki.

## CZĘŚĆ III

### ZADANIE 7
    Zapoznaj się z funkcją służącą do wykonania testu niezależności chi-kwadrat.

### ZADANIE 8
    Korzystając z funkcji poznanej w zadaniu 7. zweryfikuj hipotezę, że stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w pierwszym badanym okresie nie zależy od zajmowanego stanowiska. Przyjmij poziom istotności 0.01. Wynik testu porównaj z wynikiem uzyskanym w zadaniu 6. Zaprezentuj reszty wyznaczane w teście na wykresie asocjacyjnym i dokonaj jego interpretacji.

### ZADANIE 9
    Zapoznaj się z funkcję służącą do generowania realizacji wektorów losowych z rozkładu wielomianowego, a następnie korzystając z niej przeprowadź symulacje w celu oszacowania mocy testu Fishera oraz mocy testu chi-kwadrat Pearsona, generując dane z tabeli 2×2, w której p11 = 1/40, p12 = 3/40, p21 = 19/40, p22 = 17/40. Symulacje wykonaj dla n = 50, n = 100 oraz n = 1000. Sformułuj wnioski.

### ZADANIE 10
    Napisz funkcję, która dla danych z tablicy dwudzielczej oblicza wartość poziomu krytycznego w teście niezależnosci opartym na ilorazie wiarogodnosci. Korzystając z napisanej funkcji,wykonaj test dla danych przeanalizowanych w zadaniu 8.

## CZĘŚĆ IV i V

### ZADANIE 11
    Przeprowadzone wsród brytyjskich mężczyzn badanie trwające 20 lat wykazało, że odsetek zmarłych (na rok) z powodu raka płuc wynosił 0,00140 wsród osób palących papierosy i 0,00010 wsród osób niepalących. Odsetek zmarłych z powodu choroby niedokrwiennej serca wynosił 0,00669 dla palaczy i 0,00413 dla osób niepalących. Opisz związek pomiędzy paleniem papierosów a śmiercią z powodu raka płuc oraz związek pomiędzy paleniem papierosów a śmiercią z powodu choroby serca. Skorzystaj z różnicy proporcji, ryzyka względnego i ilorazu szans. Zinterpretuj wartości. Związek której pary zmiennych jest silniejszy?

### ZADANIE 12
    Tabela 1 przedstawia wyniki dotyczące śmiertelności kierowców i pasażerów w wypadkach samochodowych na Florydzie w 2008 roku, w zależności od tego, czy osoba miała zapięty pas bezpieczeństwa czy nie.

<div align="center">

| | Śmiertelny | Nieśmiertelny |
| ----------- | ----------- | ----------- |
| **Bez pasów** | 1085 | 55 623 |
| **Z pasami** | 703 | 441 239 |

</div> 
<br>
<div align="center">
Table 1: Wyniki dotyczące śmiertelności w wypadkach samochodowych na Florydzie w 2008 roku.
</div>

        a) Oszacuj warunkowe prawdopodobieństwo śmierci w wypadku ze względu na drugą zmienną, tj. dla kierowców i pasażerów, którzy użyli pasa bezpieczeństwa oraz dla kierowców i pasażerów, który nie użyli pasa bezpieczeństwa.
        b) Oszacuj warunkowe prawdopodobieństwo użycia pasa bezpieczeństwa ze względu na drugą zmienną, tj. dla kierowców i pasażerów ze śmiertelnymi obrażeniami oraz dla kierowców i pasażerów, którzy przeżyli wypadek.
        c) Jaki jest najbardziej naturalny wybór dla zmiennej objaśnianej w tym badaniu? Dla takiego wyboru wyznacz i zinterpretuj różnice proporcji, ryzyko względne oraz iloraz szans. Dlaczego wartości ryzyka względnego i ilorazu szans przyjmują zbliżone wartości?

### ZADANIE 13
    Oblicz wartości odpowiednich miar współzmienności (współczynnik tau lub współczynnik gamma) dla zmiennych:
        • stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w pierwszym badanym okresie i zajmowane stanowisko,
        • stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w pierwszym badanym okresie i staż pracy,
        • zajmowane stanowisko i staż pracy.

### ZADANIE 14
    Na podstawie informacji przedstawionych na wykładzie napisz własną funkcję do przeprowadzania analizy korespondencji. Funkcja powinna przyjmować jako argument tablicę dwudzielczą i zwracać obliczone wartości odpowiednich wektorów i macierzy, współrzędnych punktów oraz odpowiedni wykres. Korzystając z napisanej funkcji wykonaj analizę korespondencji dla danych dotyczących stopnia zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w pierwszym badanym okresie oraz stażu pracy.

-------------------------------------------------------

## ZADANIA DODATKOWE

### ZADANIE *1
    Napisz funkcję, która dla dwóch wektorów danych oblicza wartość poziomu krytycznego (p-value) w teście opartym na korelacji odległości. Następnie dla wygenerowanych danych zweryfikuj hipotezę o niezależności przy użyciu napisanej funkcji.

### ZADANIE *2
    Dla zadanych π1 oraz π2 pokaż, że wartość ryzyka względnego (RR) nie jest bardziej oddalona od wartości 1 (wartość odpowiadająca niezależności) niż wartość odpowiadającego ilorazu szans (OR).

### ZADANIE *3
    Niech D oznacza posiadanie pewnej choroby, a E pozostawanie wystawionym na pewny czynnik ryzyka. W badaniach epidemiologicznych definuje się miarę AR nazywaną ryzykiem przypisanym (ang. attributable risk).
        a) Niech P(E′) = 1−P(E), wówczas AR = [P(D)−P(D|E′)]/P(D). Wyjaśnij interpretację miary na podstawie wzoru.
        b) Pokaż, że AR ma związek z ryzykiem względnym, tzn.:
                AR=[P(E)(RR−1)]/[1+P(E)(RR−1)]